In [1]:
import pickle
import os.path
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

def gsheet_api_check(SCOPES):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)    
            
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                '../config/credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)        
            
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)    
            
    return creds


from googleapiclient.discovery import build

def pull_sheet_data(SCOPES,SPREADSHEET_ID,RANGE_NAME):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=RANGE_NAME).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
        return None
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=RANGE_NAME).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data

In [5]:
import pandas as pd

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SPREADSHEET_ID = '1owMKO4jBhn2ASeVq6bfDqhNwaGShYCZ1_gkrgGSg2A8'
RANGE_NAME = 'Sheet1'

data = pull_sheet_data(SCOPES,SPREADSHEET_ID,RANGE_NAME)
df = pd.DataFrame(data[1:], columns=data[0])

df

COMPLETE: Data copied


,Admission Date,LSAT,GPA,Date Scholarship Received,Scholarship,Named Scholarship?,Conditional Scholarship?
0,2021-01-28,120,3.8,2021-02-05,35,No,No
1,2021-01-13,155,2.98,2021-01-27,60,No,Yes
2,2021-03-24,139,3.7,2021-04-26,45,No,No
3,2020-12-29,171,3.85,2021-01-06,150,Yes,No
